In [1]:
from pyspark.sql import Row, SQLContext
from pyspark.sql.types  import *
from pyspark.sql.functions import *
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.classification import DecisionTreeClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.ml.feature import StringIndexer

### Importing DataFrame objects from MongoDB 

In [2]:
statusDF = spark.read.format("com.mongodb.spark.sql.DefaultSource").option("uri", "mongodb://127.0.0.1/msan697.status").load()

stationDF = spark.read.format("com.mongodb.spark.sql.DefaultSource").option("uri", "mongodb://127.0.0.1/msan697.station").load()

weatherDF = spark.read.format("com.mongodb.spark.sql.DefaultSource").option("uri", "mongodb://127.0.0.1/msan697.weather").load()

tripDF = spark.read.format("com.mongodb.spark.sql.DefaultSource").option("uri", "mongodb://127.0.0.1/msan697.trip").load()

In [3]:
statusDF.show(n=5)

+--------------------+---------------+---------------+----------+-------------------+
|                 _id|bikes_available|docks_available|station_id|               time|
+--------------------+---------------+---------------+----------+-------------------+
|[5a5d49a291bd3626...|              2|             25|         2|2013/08/29 12:06:01|
|[5a5d49a291bd3626...|              2|             25|         2|2013/08/29 12:07:01|
|[5a5d49a291bd3626...|              2|             25|         2|2013/08/29 12:08:01|
|[5a5d49a291bd3626...|              2|             25|         2|2013/08/29 12:09:01|
|[5a5d49a291bd3626...|              2|             25|         2|2013/08/29 12:10:01|
+--------------------+---------------+---------------+----------+-------------------+
only showing top 5 rows



In [4]:
stationDF.show(n=5)

+--------------------+--------+----------+---+-----------------+------------------+-------------------+--------------------+
|                 _id|    city|dock_count| id|installation_date|               lat|               long|                name|
+--------------------+--------+----------+---+-----------------+------------------+-------------------+--------------------+
|[5a5d498991bd3626...|San Jose|        19|  5|         8/5/2013|         37.331415|          -121.8932|    Adobe on Almaden|
|[5a5d498991bd3626...|San Jose|        15|  6|         8/7/2013|37.336721000000004|        -121.894074|    San Pedro Square|
|[5a5d498991bd3626...|San Jose|        15|  7|         8/7/2013|         37.333798|-121.88694299999999|Paseo de San Antonio|
|[5a5d498991bd3626...|San Jose|        15|  8|         8/5/2013|         37.330165|-121.88583100000001| San Salvador at 1st|
|[5a5d498991bd3626...|San Jose|        15|  9|         8/5/2013|         37.348742|-121.89471499999999|           Japantown|


In [5]:
weatherDF.show(n=2)

+--------------------+-----------+---------+------+---------------+------------------+------------+-----------------------------+-----------------+--------------------+------------------+----------------+-------------+------------------------------+------------------+---------------------+-------------------+---------------+------------+-----------------------------+-----------------+--------------------+--------------------+----------------+--------+
|                 _id|cloud_cover|     date|events|max_dew_point_f|max_gust_speed_mph|max_humidity|max_sea_level_pressure_inches|max_temperature_f|max_visibility_miles|max_wind_Speed_mph|mean_dew_point_f|mean_humidity|mean_sea_level_pressure_inches|mean_temperature_f|mean_visibility_miles|mean_wind_speed_mph|min_dew_point_f|min_humidity|min_sea_level_pressure_inches|min_temperature_f|min_visibility_miles|precipitation_inches|wind_dir_degrees|zip_code|
+--------------------+-----------+---------+------+---------------+------------------+--

In [6]:
tripDF.show(n=5)

+--------------------+-------+--------+---------------+--------------+--------------------+----+---------------+----------------+--------------------+-----------------+--------+
|                 _id|bike_id|duration|       end_date|end_station_id|    end_station_name|  id|     start_date|start_station_id|  start_station_name|subscription_type|zip_code|
+--------------------+-------+--------+---------------+--------------+--------------------+----+---------------+----------------+--------------------+-----------------+--------+
|[5a5d4ec891bd3626...|    520|      63|8/29/2013 14:14|            66|South Van Ness at...|4576|8/29/2013 14:13|              66|South Van Ness at...|       Subscriber|   94127|
|[5a5d4ec891bd3626...|    661|      70|8/29/2013 14:43|            10|  San Jose City Hall|4607|8/29/2013 14:42|              10|  San Jose City Hall|       Subscriber|   95138|
|[5a5d4ec891bd3626...|     48|      71|8/29/2013 10:17|            27|Mountain View Cit...|4130|8/29/2013 10:1

In [7]:
statusDF.columns

['_id', 'bikes_available', 'docks_available', 'station_id', 'time']

In [8]:
tripDF.columns

['_id',
 'bike_id',
 'duration',
 'end_date',
 'end_station_id',
 'end_station_name',
 'id',
 'start_date',
 'start_station_id',
 'start_station_name',
 'subscription_type',
 'zip_code']

### Adding features 

#### Weekday/Weekend 

In [9]:
#Adding day of week column
statusDF = statusDF.withColumn('dayofweek',date_format(from_unixtime(unix_timestamp(statusDF["time"][0:10], 'yyyy/MM/dd')),'EEEE'))

In [10]:
statusDF.select('time','dayofweek').show(n=5)

+-------------------+---------+
|               time|dayofweek|
+-------------------+---------+
|2013/08/29 12:06:01| Thursday|
|2013/08/29 12:07:01| Thursday|
|2013/08/29 12:08:01| Thursday|
|2013/08/29 12:09:01| Thursday|
|2013/08/29 12:10:01| Thursday|
+-------------------+---------+
only showing top 5 rows



In [11]:
#Adding weekend column
statusDF = statusDF.withColumn("weekend", when(col('dayofweek') == 'Saturday',1).when(col('dayofweek') == 'Sunday', 1).otherwise(0))
#Adding weekday column
statusDF = statusDF.withColumn("weekday", when(col('dayofweek') == 'Saturday',0).when(col('dayofweek') == 'Sunday', 0).otherwise(1))

In [12]:
#Checking output
statusDF.select('time','dayofweek','weekend','weekday').where(statusDF.dayofweek == "Sunday").show(n=1)

+-------------------+---------+-------+-------+
|               time|dayofweek|weekend|weekday|
+-------------------+---------+-------+-------+
|2013/09/01 00:00:02|   Sunday|      1|      0|
+-------------------+---------+-------+-------+
only showing top 1 row



#### Hour of Day/Morning/Afternoon/Evening/Night 

In [13]:
#Adding hourofday column
statusDF = statusDF.withColumn('hourofday',statusDF["time"][12:2])

We define morning as the time between 5am and 12pm, afternoon between 12pm and 5pm, evening between 5pm and 11pm and night between 11pm and 5am.

In [14]:
#Adding morning column
statusDF = statusDF.withColumn("morning", when(col('hourofday').between(5,11),1).otherwise(0))
#Adding afternoon column
statusDF = statusDF.withColumn("afternoon", when(col('hourofday').between(12,16),1).otherwise(0))
#Adding evening column
statusDF = statusDF.withColumn("evening", when(col('hourofday').between(17,22),1).otherwise(0))
#Adding night column
statusDF = statusDF.withColumn("night", when(col('hourofday').between(23,24), 1).when(col('hourofday').between(0,4),1).otherwise(0))

In [15]:
statusDF.select('time','hourofday','morning','afternoon','evening','night').show(n=5)

+-------------------+---------+-------+---------+-------+-----+
|               time|hourofday|morning|afternoon|evening|night|
+-------------------+---------+-------+---------+-------+-----+
|2013/08/29 12:06:01|       12|      0|        1|      0|    0|
|2013/08/29 12:07:01|       12|      0|        1|      0|    0|
|2013/08/29 12:08:01|       12|      0|        1|      0|    0|
|2013/08/29 12:09:01|       12|      0|        1|      0|    0|
|2013/08/29 12:10:01|       12|      0|        1|      0|    0|
+-------------------+---------+-------+---------+-------+-----+
only showing top 5 rows



#### Month

In [16]:
#Adding month column
statusDF = statusDF.withColumn('month',month(from_unixtime(unix_timestamp(statusDF["time"][0:10], 'yyyy/MM/dd'))))
# Adding year column
statusDF = statusDF.withColumn('year',year(from_unixtime(unix_timestamp(statusDF["time"][0:10], 'yyyy/MM/dd'))))

In [17]:
statusDF.select('time','month', 'year').show(n=5)

+-------------------+-----+----+
|               time|month|year|
+-------------------+-----+----+
|2013/08/29 12:06:01|    8|2013|
|2013/08/29 12:07:01|    8|2013|
|2013/08/29 12:08:01|    8|2013|
|2013/08/29 12:09:01|    8|2013|
|2013/08/29 12:10:01|    8|2013|
+-------------------+-----+----+
only showing top 5 rows



In [19]:
#Features so far
statusDF.select('station_id', 'weekend', 'weekday', 'morning', 'afternoon', 'evening', 'night', 'hourofday', 'month','year').show(n=10)

+----------+-------+-------+-------+---------+-------+-----+---------+-----+----+
|station_id|weekend|weekday|morning|afternoon|evening|night|hourofday|month|year|
+----------+-------+-------+-------+---------+-------+-----+---------+-----+----+
|         2|      0|      1|      0|        1|      0|    0|       12|    8|2013|
|         2|      0|      1|      0|        1|      0|    0|       12|    8|2013|
|         2|      0|      1|      0|        1|      0|    0|       12|    8|2013|
|         2|      0|      1|      0|        1|      0|    0|       12|    8|2013|
|         2|      0|      1|      0|        1|      0|    0|       12|    8|2013|
|         2|      0|      1|      0|        1|      0|    0|       12|    8|2013|
|         2|      0|      1|      0|        1|      0|    0|       12|    8|2013|
|         2|      0|      1|      0|        1|      0|    0|       12|    8|2013|
|         2|      0|      1|      0|        1|      0|    0|       12|    8|2013|
|         2|    

##### Weather Table

In [20]:
weatherDF = weatherDF.withColumn("events", when(col('events') == 'Fog', 1).\
                                 when(col('events').like ('%ain'),2).\
                                 when(col('events') == 'Fog-Rain',3).\
                                 when(col('events') == 'Rain-Thunderstorm',4).\
                                 otherwise(0))

#### Station Table

In [21]:
# add age of the docks
stationDF = stationDF.withColumn('age', \
               datediff(from_unixtime(unix_timestamp(date_format(current_date(), "M/d/y"), 'MM/dd/yyy')),\
                              from_unixtime(unix_timestamp(stationDF['installation_date'], 'MM/dd/yyy'))))

In [22]:
stationDF.take(1)

[Row(_id=Row(oid='5a5d498991bd3626222c4b61'), city='San Jose', dock_count=19, id=5, installation_date='8/5/2013', lat=37.331415, long=-121.8932, name='Adobe on Almaden', age=1626)]

##### Trip Table

In [27]:
tripDF = tripDF.withColumn('start_date', concat(col('start_date'),lit(':00'))).withColumn('end_date', concat(col('end_date'),lit(':00')))

In [28]:
tripDF = tripDF.withColumn('dayofweek',date_format(from_unixtime(unix_timestamp('start_date', 'MM/dd/yyy HH:mm:ss')),'EEEE'))\
.withColumn("weekend", when(col('dayofweek') == 'Saturday',1).when(col('dayofweek') == 'Sunday', 1).otherwise(0))\
.withColumn("weekday", when(col('dayofweek') == 'Saturday',0).when(col('dayofweek') == 'Sunday', 0).otherwise(1))\
.withColumn('hourofday',hour(from_unixtime(unix_timestamp('start_date', 'MM/dd/yyy HH:mm:ss'))))\
.withColumn('month',month(from_unixtime(unix_timestamp('start_date', 'MM/dd/yyy HH:mm:ss'))))\
.withColumn('year',year(from_unixtime(unix_timestamp('start_date', 'MM/dd/yyy HH:mm:ss'))))

In [29]:
tripDF.take(1)

[Row(_id=Row(oid='5a5d4ec891bd36262276bf2a'), bike_id=520, duration=63, end_date='8/29/2013 14:14:00', end_station_id=66, end_station_name='South Van Ness at Market', id=4576, start_date='8/29/2013 14:13:00', start_station_id=66, start_station_name='South Van Ness at Market', subscription_type='Subscriber', zip_code='94127', dayofweek='Thursday', weekend=0, weekday=1, hourofday=14, month=8, year=2013)]

In [30]:
outgoing_bikesDF = tripDF.groupBy('zip_code', 'start_station_id', 'hourofday', 'weekend', 'weekday', 'month', 'year').agg(count('*').alias('outgoing_bikes_count'))

In [31]:
incoming_bikesDF = tripDF.groupBy('zip_code', 'end_station_id', 'hourofday' , 'weekend', 'weekday', 'month', 'year').agg(count('*').alias('incoming_bikes_count'))

In [32]:
sqlContext.sql("drop table if exists outgoing_bikesDF")
sqlContext.sql("drop table if exists incoming_bikesDF")

DataFrame[]

In [33]:
outgoing_bikesDF.write.saveAsTable("outgoing_bikesDF")
incoming_bikesDF.write.saveAsTable("incoming_bikesDF")

In [34]:
incoming_bikesDF.take(1)

[Row(zip_code='94115', end_station_id=50, hourofday=22, weekend=0, weekday=1, month=8, year=2013, incoming_bikes_count=1)]

In [35]:
joined_df = sqlContext.sql("""
SELECT outgoing_bikesDF.zip_code AS zip_code,
start_station_id,
end_station_id,
outgoing_bikesDF.hourofday AS hourofday,
outgoing_bikesDF.weekend AS weekend,
outgoing_bikesDF.weekday AS weekday,
outgoing_bikesDF.month AS month,
outgoing_bikesDF.year AS year,
outgoing_bikes_count,
incoming_bikes_count
FROM outgoing_bikesDF LEFT JOIN incoming_bikesDF 
on outgoing_bikesDF.start_station_id = incoming_bikesDF.end_station_id
and outgoing_bikesDF.hourofday = incoming_bikesDF.hourofday
and outgoing_bikesDF.weekend = incoming_bikesDF.weekend
and outgoing_bikesDF.weekday = incoming_bikesDF.weekday
and outgoing_bikesDF.month = incoming_bikesDF.month
and outgoing_bikesDF.year = incoming_bikesDF.year
""")

In [36]:
statusDF.take(1)

[Row(_id=Row(oid='5a5d49a291bd3626222c4ba7'), bikes_available=2, docks_available=25, station_id=2, time='2013/08/29 12:06:01', dayofweek='Thursday', weekend=0, weekday=1, hourofday='12', morning=0, afternoon=1, evening=0, night=0, month=8, year=2013)]

In [37]:
statusDF = statusDF.withColumn("hourofday", statusDF["hourofday"].cast(IntegerType()))

In [38]:
stationDF.join(joined_df, stationDF.id == joined_df.end_station_id, how='left').take(1)

[Row(_id=Row(oid='5a5d498991bd3626222c4b61'), city='San Jose', dock_count=19, id=5, installation_date='8/5/2013', lat=37.331415, long=-121.8932, name='Adobe on Almaden', age=1626, zip_code='94002', start_station_id=5, end_station_id=5, hourofday=15, weekend=0, weekday=1, month=1, year=2014, outgoing_bikes_count=5, incoming_bikes_count=1)]

In [39]:
joined_df2 = stationDF.join(joined_df, stationDF.id == joined_df.end_station_id, how='left')

In [40]:
joined_df2.take(1), statusDF.take(1)

([Row(_id=Row(oid='5a5d498991bd3626222c4b61'), city='San Jose', dock_count=19, id=5, installation_date='8/5/2013', lat=37.331415, long=-121.8932, name='Adobe on Almaden', age=1626, zip_code='94002', start_station_id=5, end_station_id=5, hourofday=15, weekend=0, weekday=1, month=1, year=2014, outgoing_bikes_count=5, incoming_bikes_count=1)],
 [Row(_id=Row(oid='5a5d49a291bd3626222c4ba7'), bikes_available=2, docks_available=25, station_id=2, time='2013/08/29 12:06:01', dayofweek='Thursday', weekend=0, weekday=1, hourofday=12, morning=0, afternoon=1, evening=0, night=0, month=8, year=2013)])

In [41]:
statusDF.write.saveAsTable('statusDF')

In [56]:
sqlContext.sql("select count(*) from statusDF where year is null").show()

+--------+
|count(1)|
+--------+
|54989832|
+--------+



In [ ]:
statusDF_avg = statusDF.groupBy('station_id', 'weekend', 'weekday', 'hourofday', 'month', 'year')\
.agg(avg('bikes_available').alias('avg_bikes_available'), avg('docks_available').alias('avg_docks_available'))

In [124]:
joined_df3 = joined_df2.join(statusDF, (joined_df2.id == statusDF.station_id) \
                             & (joined_df2.hourofday == statusDF.hourofday) \
                            & (joined_df2.weekend == statusDF.weekend)\
                            & (joined_df2.weekday == statusDF.weekday)\
                            & (joined_df2.month == statusDF.month)\
                            & (joined_df2.year == statusDF.year))

In [ ]:
joined_df3.take(1)